In [1]:
import numpy as np
import pandas as pd
from pandas import concat
import matplotlib.pyplot as plt
from functools import reduce
import importlib
import datetime as dt
from datetime import datetime
from math import floor
from bayes_opt import BayesianOptimization
import seaborn as sns
from math import sqrt


#tensorflow &keras
import tensorflow as tf
from tensorflow.keras.layers import GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras import optimizers
from tensorflow.keras import initializers
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from keras.layers import concatenate
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier
from keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.callbacks import EarlyStopping
from keras.layers import LeakyReLU
LeakyReLU = LeakyReLU(alpha=0.1)

#sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from sklearn import metrics
from sklearn import preprocessing as prep
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import accuracy_score, make_scorer
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import make_scorer, accuracy_score, mean_absolute_error
from sklearn.metrics import mean_squared_error as MSE
from hyperopt import hp, fmin, tpe

import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)

In [2]:
date_format="%m/%d/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
dat = pd.read_csv("C:/Users/ciara/OneDrive/Documents/BM_data.csv", index_col="SettlementPeriod", parse_dates=True, date_parser=date_parse)

# dat = pd.read_csv("C:/Users/ciara/Downloads/TimerSeries1.csv", index_col="DeliveryPeriod", parse_dates=True, date_parser=date_parse)
dat = dat.drop(["index"], axis=1)
dat=pd.DataFrame(dat)
dat=dat.bfill(axis ='rows')
dat=dat.ffill(axis ='rows')
dat=dat._get_numeric_data()

In [3]:
Y=dat.iloc[:, 0:16]
X=dat.iloc[:,16:]
X_train=X.iloc[:43826,:]
Y_train=Y.iloc[:43826,:]
X_test=X.iloc[43826:43827,:]
Y_test=Y.iloc[43826:43827,:]

In [4]:
# rnn_train1=X_train[["lag_-3x", "lag_-4x","lag_-5x","lag_-6x","lag_-7x", "lag_-8x","lag_-9x","lag_-10x","lag_-11x", "lag_-12x","lag_-13x","lag_-14x","lag_-15x", "lag_-16x","lag_-17x","lag_-18x","lag_-19x", "lag_-20x","lag_-21x","lag_-22x","lag_-23x", "lag_-24x","lag_-25x","lag_-26x","lag_-27x", "lag_-28x","lag_-29x","lag_-30x","lag_-31x", "lag_-32x","lag_-33x","lag_-34x","lag_-35x", "lag_-36x","lag_-37x","lag_-38x","lag_-39x", "lag_-40x","lag_-41x","lag_-42x","lag_-43x", "lag_-44x","lag_-45x","lag_-46x","lag_-47x", "lag_-48x","lag_-49x","lag_-50x","lag_-51x"]]
rnn_train1=X_train[["lag_-3x1", "lag_-4x1","lag_-5x1","lag_-6x1","lag_-7x1", "lag_-8x1","lag_-9x1","lag_-10x1","lag_-11x1", "lag_-12x1","lag_-13x1","lag_-14x1","lag_-15x1", "lag_-16x1","lag_-17x1","lag_-18x1","lag_-19x1", "lag_-20x1","lag_-21x1","lag_-22x1","lag_-23x1", "lag_-24x1","lag_-25x1","lag_-26x1","lag_-27x1", "lag_-28x1","lag_-29x1","lag_-30x1","lag_-31x1", "lag_-32x1","lag_-33x1","lag_-34x1","lag_-35x1", "lag_-36x1","lag_-37x1","lag_-38x1","lag_-39x1", "lag_-40x1","lag_-41x1","lag_-42x1","lag_-43x1", "lag_-44x1","lag_-45x1","lag_-46x1","lag_-47x1", "lag_-48x1","lag_-49x1","lag_-50x1"]]
rnn_train2=X_train[["lag_-3x2", "lag_-4x2","lag_-5x2","lag_-6x2","lag_-7x2", "lag_-8x2","lag_-9x2","lag_-10x2","lag_-11x2", "lag_-12x2","lag_-13x2","lag_-14x2","lag_-15x2", "lag_-16x2","lag_-17x2","lag_-18x2","lag_-19x2", "lag_-20x2","lag_-21x2","lag_-22x2","lag_-23x2", "lag_-24x2","lag_-25x2","lag_-26x2","lag_-27x2", "lag_-28x2","lag_-29x2","lag_-30x2","lag_-31x2", "lag_-32x2","lag_-33x2","lag_-34x2","lag_-35x2", "lag_-36x2","lag_-37x2","lag_-38x2","lag_-39x2", "lag_-40x2","lag_-41x2","lag_-42x2","lag_-43x2", "lag_-44x2","lag_-45x2","lag_-46x2","lag_-47x2", "lag_-48x2","lag_-49x2","lag_-50x2"]]
rnn_train3=X_train[["lag_-2x3","lag_-3x3", "lag_-4x3","lag_-5x3","lag_-6x3","lag_-7x3", "lag_-8x3","lag_-9x3","lag_-10x3","lag_-11x3", "lag_-12x3","lag_-13x3","lag_-14x3","lag_-15x3", "lag_-16x3","lag_-17x3","lag_-18x3","lag_-19x3", "lag_-20x3","lag_-21x3","lag_-22x3","lag_-23x3", "lag_-24x3","lag_-25x3","lag_-26x3","lag_-27x3", "lag_-28x3","lag_-29x3","lag_-30x3","lag_-31x3", "lag_-32x3","lag_-33x3","lag_-34x3","lag_-35x3", "lag_-36x3","lag_-37x3","lag_-38x3","lag_-39x3", "lag_-40x3","lag_-41x3","lag_-42x3","lag_-43x3", "lag_-44x3","lag_-45x3","lag_-46x3","lag_-47x3", "lag_-48x3","lag_-49x3"]]
rnn_train4=X_train[["lag_0x6","lag_-1x6","lag_-2x6","lag_-3x6", "lag_-4x6","lag_-5x6","lag_-6x6","lag_-7x6", "lag_-8x6","lag_-9x6","lag_-10x6","lag_-11x6", "lag_-12x6","lag_-13x6","lag_-14x6","lag_-15x6", "lag_-16x6","lag_-17x6","lag_-18x6","lag_-19x6", "lag_-20x6","lag_-21x6","lag_-22x6","lag_-23x6", "lag_-24x6","lag_-25x6","lag_-26x6","lag_-27x6", "lag_-28x6","lag_-29x6","lag_-30x6","lag_-31x6", "lag_-32x6","lag_-33x6","lag_-34x6","lag_-35x6", "lag_-36x6","lag_-37x6","lag_-38x6","lag_-39x6", "lag_-40x6","lag_-41x6","lag_-42x6","lag_-43x6", "lag_-44x6","lag_-45x6","lag_-46x6","lag_-47x6"]]
rnn_train5=X_train[["lag_-2x12","lag_-3x12", "lag_-4x12","lag_-5x12","lag_-6x12","lag_-7x12", "lag_-8x12","lag_-9x12","lag_-10x12","lag_-11x12", "lag_-12x12","lag_-13x12","lag_-14x12","lag_-15x12", "lag_-16x12","lag_-17x12","lag_-18x12","lag_-19x12", "lag_-20x12","lag_-21x12","lag_-22x12","lag_-23x12", "lag_-24x12","lag_-25x12","lag_-26x12","lag_-27x12", "lag_-28x12","lag_-29x12","lag_-30x12","lag_-31x12", "lag_-32x12","lag_-33x12","lag_-34x12","lag_-35x12", "lag_-36x12","lag_-37x12","lag_-38x12","lag_-39x12", "lag_-40x12","lag_-41x12","lag_-42x12","lag_-43x12", "lag_-44x12","lag_-45x12","lag_-46x12","lag_-47x12", "lag_-48x12","lag_-49x12"]]

rnn_test1=X_test[["lag_-3x1", "lag_-4x1","lag_-5x1","lag_-6x1","lag_-7x1", "lag_-8x1","lag_-9x1","lag_-10x1","lag_-11x1", "lag_-12x1","lag_-13x1","lag_-14x1","lag_-15x1", "lag_-16x1","lag_-17x1","lag_-18x1","lag_-19x1", "lag_-20x1","lag_-21x1","lag_-22x1","lag_-23x1", "lag_-24x1","lag_-25x1","lag_-26x1","lag_-27x1", "lag_-28x1","lag_-29x1","lag_-30x1","lag_-31x1", "lag_-32x1","lag_-33x1","lag_-34x1","lag_-35x1", "lag_-36x1","lag_-37x1","lag_-38x1","lag_-39x1", "lag_-40x1","lag_-41x1","lag_-42x1","lag_-43x1", "lag_-44x1","lag_-45x1","lag_-46x1","lag_-47x1", "lag_-48x1","lag_-49x1","lag_-50x1"]]
rnn_test2=X_test[["lag_-3x2", "lag_-4x2","lag_-5x2","lag_-6x2","lag_-7x2", "lag_-8x2","lag_-9x2","lag_-10x2","lag_-11x2", "lag_-12x2","lag_-13x2","lag_-14x2","lag_-15x2", "lag_-16x2","lag_-17x2","lag_-18x2","lag_-19x2", "lag_-20x2","lag_-21x2","lag_-22x2","lag_-23x2", "lag_-24x2","lag_-25x2","lag_-26x2","lag_-27x2", "lag_-28x2","lag_-29x2","lag_-30x2","lag_-31x2", "lag_-32x2","lag_-33x2","lag_-34x2","lag_-35x2", "lag_-36x2","lag_-37x2","lag_-38x2","lag_-39x2", "lag_-40x2","lag_-41x2","lag_-42x2","lag_-43x2", "lag_-44x2","lag_-45x2","lag_-46x2","lag_-47x2", "lag_-48x2","lag_-49x2","lag_-50x2"]]
rnn_test3=X_test[["lag_-2x3","lag_-3x3", "lag_-4x3","lag_-5x3","lag_-6x3","lag_-7x3", "lag_-8x3","lag_-9x3","lag_-10x3","lag_-11x3", "lag_-12x3","lag_-13x3","lag_-14x3","lag_-15x3", "lag_-16x3","lag_-17x3","lag_-18x3","lag_-19x3", "lag_-20x3","lag_-21x3","lag_-22x3","lag_-23x3", "lag_-24x3","lag_-25x3","lag_-26x3","lag_-27x3", "lag_-28x3","lag_-29x3","lag_-30x3","lag_-31x3", "lag_-32x3","lag_-33x3","lag_-34x3","lag_-35x3", "lag_-36x3","lag_-37x3","lag_-38x3","lag_-39x3", "lag_-40x3","lag_-41x3","lag_-42x3","lag_-43x3", "lag_-44x3","lag_-45x3","lag_-46x3","lag_-47x3", "lag_-48x3","lag_-49x3"]]
rnn_test4=X_test[["lag_0x6","lag_-1x6","lag_-2x6","lag_-3x6", "lag_-4x6","lag_-5x6","lag_-6x6","lag_-7x6", "lag_-8x6","lag_-9x6","lag_-10x6","lag_-11x6", "lag_-12x6","lag_-13x6","lag_-14x6","lag_-15x6", "lag_-16x6","lag_-17x6","lag_-18x6","lag_-19x6", "lag_-20x6","lag_-21x6","lag_-22x6","lag_-23x6", "lag_-24x6","lag_-25x6","lag_-26x6","lag_-27x6", "lag_-28x6","lag_-29x6","lag_-30x6","lag_-31x6", "lag_-32x6","lag_-33x6","lag_-34x6","lag_-35x6", "lag_-36x6","lag_-37x6","lag_-38x6","lag_-39x6", "lag_-40x6","lag_-41x6","lag_-42x6","lag_-43x6", "lag_-44x6","lag_-45x6","lag_-46x6","lag_-47x6"]]
rnn_test5=X_test[["lag_-2x12","lag_-3x12", "lag_-4x12","lag_-5x12","lag_-6x12","lag_-7x12", "lag_-8x12","lag_-9x12","lag_-10x12","lag_-11x12", "lag_-12x12","lag_-13x12","lag_-14x12","lag_-15x12", "lag_-16x12","lag_-17x12","lag_-18x12","lag_-19x12", "lag_-20x12","lag_-21x12","lag_-22x12","lag_-23x12", "lag_-24x12","lag_-25x12","lag_-26x12","lag_-27x12", "lag_-28x12","lag_-29x12","lag_-30x12","lag_-31x12", "lag_-32x12","lag_-33x12","lag_-34x12","lag_-35x12", "lag_-36x12","lag_-37x12","lag_-38x12","lag_-39x12", "lag_-40x12","lag_-41x12","lag_-42x12","lag_-43x12", "lag_-44x12","lag_-45x12","lag_-46x12","lag_-47x12", "lag_-48x12","lag_-49x12"]]

rnn_Y=Y_train[["y", "lag_3y", "lag_4y", "lag_5y", "lag_6y", "lag_7y", "lag_8y", "lag_9y", "lag_10y", "lag_11y", "lag_12y", "lag_13y", "lag_14y", "lag_15y", "lag_16y", "lag_17y"]]

X_scaler1 = preprocessing.MinMaxScaler()
X_scaler2 = preprocessing.MinMaxScaler()
X_scaler3 = preprocessing.MinMaxScaler()
X_scaler4 = preprocessing.MinMaxScaler()
X_scaler5 = preprocessing.MinMaxScaler()

Y_scaler = preprocessing.MinMaxScaler()

rnn_scaled_train1 = X_scaler1.fit_transform(rnn_train1)
rnn_scaled_train2 = X_scaler2.fit_transform(rnn_train2)
rnn_scaled_train3 = X_scaler3.fit_transform(rnn_train3)
rnn_scaled_train4 = X_scaler4.fit_transform(rnn_train4)
rnn_scaled_train5 = X_scaler5.fit_transform(rnn_train5)

Y_train_Scaled   = Y_scaler.fit_transform(Y_train)

X_train_Scaled = np.hstack(
    (rnn_scaled_train1, rnn_scaled_train2, rnn_scaled_train3,
     rnn_scaled_train4, rnn_scaled_train5)
).reshape(rnn_train1.shape[0], 5, 48).transpose(0, 2, 1)

X_test_Scaled = np.hstack(
    (X_scaler1.transform(rnn_test1), X_scaler2.transform(rnn_test2),
     X_scaler3.transform(rnn_test3), X_scaler4.transform(rnn_test4),
     X_scaler5.transform(rnn_test5))
).reshape(rnn_test1.shape[0], 5, 48).transpose(0, 2, 1)

In [11]:
score_acc = make_scorer(mean_absolute_error)
mse = make_scorer(MSE, greater_is_better=False)
i_shape=(None,X_train_Scaled.shape[1], X_train_Scaled.shape[2])


In [12]:
X_train_Scaled.shape
i_shape

(None, 48, 5)

In [16]:
def create_model():
        nn = Sequential()
        for i in range(3):
            nn.add(LSTM(64, return_sequences= True, input_shape=i_shape))
#         for i in range(1):
#             nn.add(Dense(100, activation='sigmoid'))
#         if 0 > 0.5:
#             nn.add(Dropout(0.5, seed=123))
#         nn.add(Dense(100, activation='tanh'))
#         for i in range(1):
#             nn.add(Dense(10, activation='tanh'))
#         for i in range(3):
#             nn.add(Dense(10, activation='tanh'))
        nn.add(LSTM(64))
        nn.add(Dense(16))
        nn.compile(loss='mean_squared_error', optimizer='Adam', metrics=['mean_squared_error'])
        nn.build(i_shape)
        nn.summary()
        return nn

es = EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=20)
nn_hyp = KerasRegressor(build_fn=create_model, epochs=1, batch_size=16,
                         verbose=1, callbacks=[es])
nn_hyp.fit(X_train_Scaled, Y_train_Scaled, verbose=1, validation_split=0.2)

# preds_nn = pd.DataFrame(Y_scaler.inverse_transform(nn_hyp.predict(X_test_Scaled).reshape(1,16)), index=Y_test.index)
# mse = mean_squared_error(Y_test, preds_nn)
# rmse = sqrt(mean_squared_error(Y_test, preds_nn))
# mae = mean_absolute_error(Y_test, preds_nn)
# Error_1 = pd.DataFrame([mse, rmse, mae]).T
# Error_1

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
module_wrapper_5 (ModuleWrap (None, 48, 64)            17920     
_________________________________________________________________
module_wrapper_6 (ModuleWrap (None, 48, 64)            33024     
_________________________________________________________________
module_wrapper_7 (ModuleWrap (None, 48, 64)            33024     
_________________________________________________________________
module_wrapper_8 (ModuleWrap (None, 64)                33024     
_________________________________________________________________
dense_4 (Dense)              (None, 16)                1040      
Total params: 118,032
Trainable params: 118,032
Non-trainable params: 0
_________________________________________________________________
2192/2192 [==============================] - 66s 26ms/step - loss: 1.9184e-04 - mean_squared_error: 1.9184e-04 - v

In [ ]:
def create_model(neurons_0, activation_0,neurons_2, activation_2, neurons_1, activation_1,neurons_3, activation_3,
             optimizer, batch_size, epochs, layers1, layers2, layers3, dropout, dropout_rate):
        nn = Sequential()
        nn.add(Flatten(input_shape=i_shape))
        nn.add(Dense(neurons_0, input_shape=i_shape, activation=activation_0))
        for i in range(layers1):
            nn.add(Dense(neurons_1, activation=activation_1))
        if dropout > 0.2:
            nn.add(Dropout(dropout_rate, seed=123))
        for i in range(layers2):
            nn.add(Dense(neurons_2, activation=activation_2))
        for i in range(layers3):
            nn.add(Dense(neurons_3, activation=activation_3))
        nn.add(Dense(16))
        nn.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mean_squared_error'])
        return nn
    
es = EarlyStopping(monitor='mean_squared_error', mode='min', verbose=0, patience=25)
nn = KerasRegressor(build_fn=create_model,  verbose=0, callbacks=[es])
# Set hyperparameters spaces
params_nn ={
    'neurons_0': (10, 100),
    'activation_0':['relu', 'sigmoid',  'tanh'],
    'neurons_1': (10, 100),
    'activation_1':['relu', 'sigmoid',  'tanh'],
    'neurons_2': (10, 100),
    'activation_2':['relu', 'sigmoid',  'tanh'],
    'neurons_3': (10, 100),
    'activation_3':['relu', 'sigmoid',  'tanh'],
    'optimizer':['Adam'],
    'batch_size':(16, 50),
    'epochs':(10, 200),
    'layers1':(1,3),
    'layers2':(1,3),
    'layers3':(1,3),

    'dropout':(0,1),
    'dropout_rate':(0,.5)
}

# nn_bo = BayesianOptimization(create_model, params_nn, random_state=123)
# nn_bo.maximize(init_points=4, n_iter=10)
grid = RandomizedSearchCV(estimator=nn, cv=8, param_distributions=params_nn)
grid_result = grid.fit(X_train_Scaled, Y_train_Scaled)
print("Best Score: ",
      grid_result.best_score_,
      "and Best Params: ",
      grid_result.best_params_)

In [ ]:
def create_model():
    nn = Sequential()
    nn.add(Flatten(input_shape=i_shape))
    nn.add(Dense(grid_result.best_params_['neurons_0'], input_shape=i_shape, activation=grid_result.best_params_['activation_0']))
    for i in range(grid_result.best_params_['layers1']):
        nn.add(Dense(grid_result.best_params_['neurons_1'], activation=grid_result.best_params_['activation_1']))
    if grid_result.best_params_['dropout'] > 0.2:
        nn.add(Dropout(grid_result.best_params_['dropout_rate'], seed=123))
    for i in range(grid_result.best_params_['layers2']):
        nn.add(Dense(grid_result.best_params_['neurons_2'], activation=grid_result.best_params_['activation_2']))
    for i in range(grid_result.best_params_['layers3']):
        nn.add(Dense(grid_result.best_params_['neurons_3'], activation=grid_result.best_params_['activation_3']))
    nn.add(Dense(16))
    nn.compile(loss='mean_squared_error', optimizer=grid_result.best_params_['optimizer'], metrics=['mean_squared_error'])
    return nn

es = EarlyStopping(monitor='mean_squared_error', mode='min', verbose=0, patience=20)        
nn_hyp = KerasRegressor(build_fn=create_model, epochs=grid_result.best_params_['epochs'], batch_size=grid_result.best_params_['batch_size'],
                         verbose=0, callbacks=[es])
nn_hyp.fit(X_train_Scaled, Y_train_Scaled, verbose=0, validation_split=0.2)
preds_nn = pd.DataFrame(Y_scaler.inverse_transform(nn_hyp.predict(X_test_Scaled).reshape(1,16)), index=Y_test.index)
mse = mean_squared_error(Y_test, pred_nn)
rmse = sqrt(mean_squared_error(Y_test, pred_nn))
mae = mean_absolute_error(Y_test, pred_nn)
Error_1 = pd.DataFrame([mse, rmse, mae]).T

In [ ]:
def create_model():
    nn = Sequential()
    nn.add(Flatten(input_shape=i_shape))
    nn.add(Dense(grid_result.best_params_['neurons_0'], input_shape=i_shape, activation=grid_result.best_params_['activation_0']))
    for i in range(grid_result.best_params_['layers1']):
        nn.add(Dense(grid_result.best_params_['neurons_1'], activation=grid_result.best_params_['activation_1']))
    if grid_result.best_params_['dropout'] > 0.2:
        nn.add(Dropout(grid_result.best_params_['dropout_rate'], seed=123))
    for i in range(grid_result.best_params_['layers2']):
        nn.add(Dense(grid_result.best_params_['neurons_2'], activation=grid_result.best_params_['activation_2']))
    for i in range(grid_result.best_params_['layers3']):
        nn.add(Dense(grid_result.best_params_['neurons_3'], activation=grid_result.best_params_['activation_3']))
    nn.add(Dense(16))
    nn.compile(loss='mean_squared_error', optimizer=grid_result.best_params_['optimizer'], metrics=['mean_squared_error'])
    return nn


es = EarlyStopping(monitor='mean_squared_error', mode='min', verbose=0, patience=20)        
nn_hyp = KerasRegressor(build_fn=create_model, epochs=grid_result.best_params_['epochs'], batch_size=grid_result.best_params_['batch_size'],
                         verbose=0, callbacks=[es])
nn_hyp.fit(X_train_Scaled, Y_train_Scaled, verbose=0, validation_split=0.2)
preds_nn = pd.DataFrame(Y_scaler.inverse_transform(nn_hyp.predict(X_test_Scaled).reshape(1,16)), index=Y_test.index)
mse = mean_squared_error(Y_test, preds_nn)
rmse = sqrt(mean_squared_error(Y_test, preds_nn))
mae = mean_absolute_error(Y_test, preds_nn)

In [ ]:
Errors_000=[]
n=41000
a=0
for i in range(0, 10, 1):
    X=dat.iloc[a+i:, 16:]
    Y=dat.iloc[a+i:, :16]
    
    X_train=X.iloc[:n,:]
    Y_train=Y.iloc[:n,:]
    X_test=X.iloc[n:n+1,:]
    Y_test=Y.iloc[n:n+1,:]

    # rnn_train1=X_train[["lag_-3x", "lag_-4x","lag_-5x","lag_-6x","lag_-7x", "lag_-8x","lag_-9x","lag_-10x","lag_-11x", "lag_-12x","lag_-13x","lag_-14x","lag_-15x", "lag_-16x","lag_-17x","lag_-18x","lag_-19x", "lag_-20x","lag_-21x","lag_-22x","lag_-23x", "lag_-24x","lag_-25x","lag_-26x","lag_-27x", "lag_-28x","lag_-29x","lag_-30x","lag_-31x", "lag_-32x","lag_-33x","lag_-34x","lag_-35x", "lag_-36x","lag_-37x","lag_-38x","lag_-39x", "lag_-40x","lag_-41x","lag_-42x","lag_-43x", "lag_-44x","lag_-45x","lag_-46x","lag_-47x", "lag_-48x","lag_-49x","lag_-50x","lag_-51x"]]
    rnn_train1=X_train[["lag_-3x1", "lag_-4x1","lag_-5x1","lag_-6x1","lag_-7x1", "lag_-8x1","lag_-9x1","lag_-10x1","lag_-11x1", "lag_-12x1","lag_-13x1","lag_-14x1","lag_-15x1", "lag_-16x1","lag_-17x1","lag_-18x1","lag_-19x1", "lag_-20x1","lag_-21x1","lag_-22x1","lag_-23x1", "lag_-24x1","lag_-25x1","lag_-26x1","lag_-27x1", "lag_-28x1","lag_-29x1","lag_-30x1","lag_-31x1", "lag_-32x1","lag_-33x1","lag_-34x1","lag_-35x1", "lag_-36x1","lag_-37x1","lag_-38x1","lag_-39x1", "lag_-40x1","lag_-41x1","lag_-42x1","lag_-43x1", "lag_-44x1","lag_-45x1","lag_-46x1","lag_-47x1", "lag_-48x1","lag_-49x1","lag_-50x1"]]
    rnn_train2=X_train[["lag_-3x2", "lag_-4x2","lag_-5x2","lag_-6x2","lag_-7x2", "lag_-8x2","lag_-9x2","lag_-10x2","lag_-11x2", "lag_-12x2","lag_-13x2","lag_-14x2","lag_-15x2", "lag_-16x2","lag_-17x2","lag_-18x2","lag_-19x2", "lag_-20x2","lag_-21x2","lag_-22x2","lag_-23x2", "lag_-24x2","lag_-25x2","lag_-26x2","lag_-27x2", "lag_-28x2","lag_-29x2","lag_-30x2","lag_-31x2", "lag_-32x2","lag_-33x2","lag_-34x2","lag_-35x2", "lag_-36x2","lag_-37x2","lag_-38x2","lag_-39x2", "lag_-40x2","lag_-41x2","lag_-42x2","lag_-43x2", "lag_-44x2","lag_-45x2","lag_-46x2","lag_-47x2", "lag_-48x2","lag_-49x2","lag_-50x2"]]
    rnn_train3=X_train[["lag_-2x3","lag_-3x3", "lag_-4x3","lag_-5x3","lag_-6x3","lag_-7x3", "lag_-8x3","lag_-9x3","lag_-10x3","lag_-11x3", "lag_-12x3","lag_-13x3","lag_-14x3","lag_-15x3", "lag_-16x3","lag_-17x3","lag_-18x3","lag_-19x3", "lag_-20x3","lag_-21x3","lag_-22x3","lag_-23x3", "lag_-24x3","lag_-25x3","lag_-26x3","lag_-27x3", "lag_-28x3","lag_-29x3","lag_-30x3","lag_-31x3", "lag_-32x3","lag_-33x3","lag_-34x3","lag_-35x3", "lag_-36x3","lag_-37x3","lag_-38x3","lag_-39x3", "lag_-40x3","lag_-41x3","lag_-42x3","lag_-43x3", "lag_-44x3","lag_-45x3","lag_-46x3","lag_-47x3", "lag_-48x3","lag_-49x3"]]
    rnn_train4=X_train[["lag_0x6","lag_-1x6","lag_-2x6","lag_-3x6", "lag_-4x6","lag_-5x6","lag_-6x6","lag_-7x6", "lag_-8x6","lag_-9x6","lag_-10x6","lag_-11x6", "lag_-12x6","lag_-13x6","lag_-14x6","lag_-15x6", "lag_-16x6","lag_-17x6","lag_-18x6","lag_-19x6", "lag_-20x6","lag_-21x6","lag_-22x6","lag_-23x6", "lag_-24x6","lag_-25x6","lag_-26x6","lag_-27x6", "lag_-28x6","lag_-29x6","lag_-30x6","lag_-31x6", "lag_-32x6","lag_-33x6","lag_-34x6","lag_-35x6", "lag_-36x6","lag_-37x6","lag_-38x6","lag_-39x6", "lag_-40x6","lag_-41x6","lag_-42x6","lag_-43x6", "lag_-44x6","lag_-45x6","lag_-46x6","lag_-47x6"]]
    rnn_train5=X_train[["lag_-2x12","lag_-3x12", "lag_-4x12","lag_-5x12","lag_-6x12","lag_-7x12", "lag_-8x12","lag_-9x12","lag_-10x12","lag_-11x12", "lag_-12x12","lag_-13x12","lag_-14x12","lag_-15x12", "lag_-16x12","lag_-17x12","lag_-18x12","lag_-19x12", "lag_-20x12","lag_-21x12","lag_-22x12","lag_-23x12", "lag_-24x12","lag_-25x12","lag_-26x12","lag_-27x12", "lag_-28x12","lag_-29x12","lag_-30x12","lag_-31x12", "lag_-32x12","lag_-33x12","lag_-34x12","lag_-35x12", "lag_-36x12","lag_-37x12","lag_-38x12","lag_-39x12", "lag_-40x12","lag_-41x12","lag_-42x12","lag_-43x12", "lag_-44x12","lag_-45x12","lag_-46x12","lag_-47x12", "lag_-48x12","lag_-49x12"]]

    rnn_test1=X_test[["lag_-3x1", "lag_-4x1","lag_-5x1","lag_-6x1","lag_-7x1", "lag_-8x1","lag_-9x1","lag_-10x1","lag_-11x1", "lag_-12x1","lag_-13x1","lag_-14x1","lag_-15x1", "lag_-16x1","lag_-17x1","lag_-18x1","lag_-19x1", "lag_-20x1","lag_-21x1","lag_-22x1","lag_-23x1", "lag_-24x1","lag_-25x1","lag_-26x1","lag_-27x1", "lag_-28x1","lag_-29x1","lag_-30x1","lag_-31x1", "lag_-32x1","lag_-33x1","lag_-34x1","lag_-35x1", "lag_-36x1","lag_-37x1","lag_-38x1","lag_-39x1", "lag_-40x1","lag_-41x1","lag_-42x1","lag_-43x1", "lag_-44x1","lag_-45x1","lag_-46x1","lag_-47x1", "lag_-48x1","lag_-49x1","lag_-50x1"]]
    rnn_test2=X_test[["lag_-3x2", "lag_-4x2","lag_-5x2","lag_-6x2","lag_-7x2", "lag_-8x2","lag_-9x2","lag_-10x2","lag_-11x2", "lag_-12x2","lag_-13x2","lag_-14x2","lag_-15x2", "lag_-16x2","lag_-17x2","lag_-18x2","lag_-19x2", "lag_-20x2","lag_-21x2","lag_-22x2","lag_-23x2", "lag_-24x2","lag_-25x2","lag_-26x2","lag_-27x2", "lag_-28x2","lag_-29x2","lag_-30x2","lag_-31x2", "lag_-32x2","lag_-33x2","lag_-34x2","lag_-35x2", "lag_-36x2","lag_-37x2","lag_-38x2","lag_-39x2", "lag_-40x2","lag_-41x2","lag_-42x2","lag_-43x2", "lag_-44x2","lag_-45x2","lag_-46x2","lag_-47x2", "lag_-48x2","lag_-49x2","lag_-50x2"]]
    rnn_test3=X_test[["lag_-2x3","lag_-3x3", "lag_-4x3","lag_-5x3","lag_-6x3","lag_-7x3", "lag_-8x3","lag_-9x3","lag_-10x3","lag_-11x3", "lag_-12x3","lag_-13x3","lag_-14x3","lag_-15x3", "lag_-16x3","lag_-17x3","lag_-18x3","lag_-19x3", "lag_-20x3","lag_-21x3","lag_-22x3","lag_-23x3", "lag_-24x3","lag_-25x3","lag_-26x3","lag_-27x3", "lag_-28x3","lag_-29x3","lag_-30x3","lag_-31x3", "lag_-32x3","lag_-33x3","lag_-34x3","lag_-35x3", "lag_-36x3","lag_-37x3","lag_-38x3","lag_-39x3", "lag_-40x3","lag_-41x3","lag_-42x3","lag_-43x3", "lag_-44x3","lag_-45x3","lag_-46x3","lag_-47x3", "lag_-48x3","lag_-49x3"]]
    rnn_test4=X_test[["lag_0x6","lag_-1x6","lag_-2x6","lag_-3x6", "lag_-4x6","lag_-5x6","lag_-6x6","lag_-7x6", "lag_-8x6","lag_-9x6","lag_-10x6","lag_-11x6", "lag_-12x6","lag_-13x6","lag_-14x6","lag_-15x6", "lag_-16x6","lag_-17x6","lag_-18x6","lag_-19x6", "lag_-20x6","lag_-21x6","lag_-22x6","lag_-23x6", "lag_-24x6","lag_-25x6","lag_-26x6","lag_-27x6", "lag_-28x6","lag_-29x6","lag_-30x6","lag_-31x6", "lag_-32x6","lag_-33x6","lag_-34x6","lag_-35x6", "lag_-36x6","lag_-37x6","lag_-38x6","lag_-39x6", "lag_-40x6","lag_-41x6","lag_-42x6","lag_-43x6", "lag_-44x6","lag_-45x6","lag_-46x6","lag_-47x6"]]
    rnn_test5=X_test[["lag_-2x12","lag_-3x12", "lag_-4x12","lag_-5x12","lag_-6x12","lag_-7x12", "lag_-8x12","lag_-9x12","lag_-10x12","lag_-11x12", "lag_-12x12","lag_-13x12","lag_-14x12","lag_-15x12", "lag_-16x12","lag_-17x12","lag_-18x12","lag_-19x12", "lag_-20x12","lag_-21x12","lag_-22x12","lag_-23x12", "lag_-24x12","lag_-25x12","lag_-26x12","lag_-27x12", "lag_-28x12","lag_-29x12","lag_-30x12","lag_-31x12", "lag_-32x12","lag_-33x12","lag_-34x12","lag_-35x12", "lag_-36x12","lag_-37x12","lag_-38x12","lag_-39x12", "lag_-40x12","lag_-41x12","lag_-42x12","lag_-43x12", "lag_-44x12","lag_-45x12","lag_-46x12","lag_-47x12", "lag_-48x12","lag_-49x12"]]

    rnn_Y=Y_train[["y", "lag_3y", "lag_4y", "lag_5y", "lag_6y", "lag_7y", "lag_8y", "lag_9y", "lag_10y", "lag_11y", "lag_12y", "lag_13y", "lag_14y", "lag_15y", "lag_16y", "lag_17y"]]

    X_scaler1 = preprocessing.MinMaxScaler()
    X_scaler2 = preprocessing.MinMaxScaler()
    X_scaler3 = preprocessing.MinMaxScaler()
    X_scaler4 = preprocessing.MinMaxScaler()
    X_scaler5 = preprocessing.MinMaxScaler()

    Y_scaler = preprocessing.MinMaxScaler()

    rnn_scaled_train1 = X_scaler1.fit_transform(rnn_train1)
    rnn_scaled_train2 = X_scaler2.fit_transform(rnn_train2)
    rnn_scaled_train3 = X_scaler3.fit_transform(rnn_train3)
    rnn_scaled_train4 = X_scaler4.fit_transform(rnn_train4)
    rnn_scaled_train5 = X_scaler5.fit_transform(rnn_train5)

    Y_train_Scaled   = Y_scaler.fit_transform(Y_train)

    X_train_Scaled = np.hstack(
        (rnn_scaled_train1, rnn_scaled_train2, rnn_scaled_train3,
         rnn_scaled_train4, rnn_scaled_train5)
    ).reshape(rnn_train1.shape[0], 5, 48).transpose(0, 2, 1)

    X_test_Scaled = np.hstack(
        (X_scaler1.transform(rnn_test1), X_scaler2.transform(rnn_test2),
         X_scaler3.transform(rnn_test3), X_scaler4.transform(rnn_test4),
         X_scaler5.transform(rnn_test5))
    ).reshape(rnn_test1.shape[0], 5, 48).transpose(0, 2, 1)
    

    def create_model():
        nn = Sequential()
        nn.add(Flatten(input_shape=i_shape))
        nn.add(Dense(grid_result.best_params_['neurons_0'], input_shape=i_shape, activation=grid_result.best_params_['activation_0']))
        for i in range(grid_result.best_params_['layers1']):
            nn.add(Dense(grid_result.best_params_['neurons_1'], activation=grid_result.best_params_['activation_1']))
        if grid_result.best_params_['dropout'] > 0.2:
            nn.add(Dropout(grid_result.best_params_['dropout_rate'], seed=123))
        for i in range(grid_result.best_params_['layers2']):
            nn.add(Dense(grid_result.best_params_['neurons_2'], activation=grid_result.best_params_['activation_2']))
        for i in range(grid_result.best_params_['layers3']):
            nn.add(Dense(grid_result.best_params_['neurons_3'], activation=grid_result.best_params_['activation_3']))
        nn.add(Dense(16))
        nn.compile(loss='mean_squared_error', optimizer=grid_result.best_params_['optimizer'], metrics=['mean_squared_error'])
        return nn
        
    es = EarlyStopping(monitor='mean_squared_error', mode='min', verbose=0, patience=20)        
    nn_hyp = KerasRegressor(build_fn=create_model, epochs=grid_result.best_params_['epochs'], batch_size=grid_result.best_params_['batch_size'],
                             verbose=0, callbacks=[es])

    nn_hyp.fit(X_train_Scaled, Y_train_Scaled, verbose=0, validation_split=0.2)
    preds_nn = pd.DataFrame(Y_scaler.inverse_transform(nn_hyp.predict(X_test_Scaled).reshape(1,16)), index=Y_test.index)

    mse = mean_squared_error(Y_test, preds_nn)
    rmse = sqrt(mean_squared_error(Y_test, preds_nn))
    mae = mean_absolute_error(Y_test, preds_nn)
    Error_i = ([mse, rmse, mae])
    Errors_000.append(Error_i)
    Errors_A=pd.DataFrame(Errors_000)

In [ ]:
Errors_A.mean()

In [ ]:
i_shape=(X_train_Scaled.shape[0],X_train_Scaled.shape[1], X_train_Scaled.shape[2])

In [ ]:
def create_model():
        nn = Sequential()
        nn.add(Flatten(input_shape=i_shape))
        for i in range(1):
            nn.add(LSTM(64, input_shape=i_shape))
#         for i in range(1):
#             nn.add(Dense(100, activation='sigmoid'))
#         if 0 > 0.5:
#             nn.add(Dropout(0.5, seed=123))
#         nn.add(Dense(100, activation='tanh'))
#         for i in range(1):
#             nn.add(Dense(10, activation='tanh'))
#         for i in range(3):
#             nn.add(Dense(10, activation='tanh'))
        nn.add(Dense(16))
        nn.compile(loss='mean_squared_error', optimizer='Adam', metrics=['mean_squared_error'])
        nn.build(i_shape)
        nn.summary()
        return nn

es = EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=20)
nn_hyp = KerasRegressor(build_fn=create_model, epochs=50, batch_size=16,
                         verbose=1, callbacks=[es])
nn_hyp.fit(X_train_Scaled, Y_train_Scaled, verbose=1, validation_split=0.2)

preds_nn = pd.DataFrame(Y_scaler.inverse_transform(nn_hyp.predict(X_test_Scaled).reshape(1,16)), index=Y_test.index)
mse = mean_squared_error(Y_test, preds_nn)
rmse = sqrt(mean_squared_error(Y_test, preds_nn))
mae = mean_absolute_error(Y_test, preds_nn)
Error_1 = pd.DataFrame([mse, rmse, mae]).T
Error_1

In [ ]:
Errors_A[2].mean()

In [ ]:
plt.plot(Errors_A[2])

In [ ]:
Errors_A.mean()